# Interpret anhand des Songtextes schätzen

## Datensatz

https://www.kaggle.com/mousehead/songlyrics

## Einleitung
Es wurde ein Datensatz gewählt, welcher unterschiedliche Interpreten und die dazugehörigen Songtexte beinhaltet. Aus diesem Datensatz soll ein Lerner generiert werden, welcher im Stande ist, Songtexte dem Interpreten zu zuweisen.

Big Data besteht zu einem Grossteil aus Textdaten, diese sind stark unstrukturiert. Um signifikante Ergebnisse bei der Analyse zu erzielen, müssen die Daten zuerst prozessiert werden. Als Werkzeug dient das Natural Language Processing, kurz NLP.

NLP unterstützt den Benutzer mit verschiedenen Tasks, unteranderem der Feature-Generierung und der Klassifizierung der Texte. NLP ist eine weit verbreitete Art der Datenanalyse und wird  für verschiedenste Anwendungen verwendet, wie Rechtschreibprüfung, personalisierte Werbung, oder Schlüsselwortsuche. Die Werkzeuge befinden sich in der verwendeten Python Natural Language Toolkit Library (NLTK).

Anwendungsgebiete des NLP sind Chatbots, Spracherkennung, oder Gefühlsanalyse, wie beispielsweise bei Facebook angewendet. Eine sehr ähnliche Anwendung zu diesem Projekt ist die Applikation Shazam, welche die Möglichkeit bietet laufende Musik zu erkennen. Ein weiteres Beispiel ist die machinelle Übersetzung, hier sind bekannte Anwendungen DeepL und der Google Übersetzer. 


## Zielsetzung
Ziel ist es eine möglichst hohe Genauigkeit bei der Ermittlung von Künstlern aus gegebenen Songtexten zu erreichen. 

## Vorgehensweise

Zu Beginn werden die Daten vorbereitet (Preprocessing), dass heisst es werden unnötige Wörter und Textteile entfernt, welche keine brauchbaren Angaben zur Eigenschaft des Textes bieten.

NLP beinhaltet zwei Arten der Sprachprozessierung, Natural Language Understanding und Natural Language Generation. In diesem Projekt wird aussschliesslich der Teil der Natural Language Understanding verwendet.

Unterteilt wird die NLP in die Tasks 
- Preprocessing
- Feature-Generierung
- Klassifikation.

Diese Tasks werden sequentiell in einer Pipeline abgearbeitet mittels einem geeigneten Klassifikator.
Zum Schluss werden verschiedene Klassifikatoren gegeneinander kreuzvalidiert.

## Preprocessing
Zuerst wird eine kurze explorative Datenanalyse durchgeführt, dann startet das Preprocessing. Die Texte werden in einzelne Wörter geteilt und betrachtet.

Drei Schritte des Preprocessing:
-	Object Standardization
-	Noise Removal
-	Lexicon Normalization

### Explorative Datenanalyse
Zuerst wird die Tabelle von unnötigen Spalten befreit.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/songdata.csv')
del df['link']
del df['song']
len(df['artist'].unique())

643

Einträge mit Künstler "Unknown" werden entfernt.

In [3]:
df = df[df['artist'] != 'Unknown']

Künstler mit nur wenigen Songs werden entfernt.

Damit die Kreuzvalidierung und das Parameter Tuning nicht zu lange dauern, sind wir hier grosszügig.

In [4]:
group = df.groupby('artist').size()
group = group[group >= 183]
#group = group[group >= 150]
df = df[[artist in group.index for artist in df['artist']]]
len(df['artist'].unique())

18

In [5]:
df.head()

,artist,text
361,Alabama,"Calling, calling all angels, oh I'm calling, c..."
362,Alabama,I thought it was forever \nI thought it would...
363,Alabama,Somewhere in the mountains......... In norther...
364,Alabama,"By now in New York City, there's snow on the g..."
365,Alabama,All my friends are asking me where I plan to s...


### Object Standardization
Strukturelle Informationen werden aus dem Datensatz mittels Regex extrahiert. 

Gelöschte Texteile:
- Zeilenumbrüche 
- Text in eckigen Klammern 
- Chorus-Bausteine
- Multiplikatoren
- Sonderzeichen
- Wiederholte Leerzeichen

In [6]:
import re
def clean(items):
    out = []
    for text in items:
        text = re.sub('[\n\r]', ' ', text)             # Zeilenumbrüche
        text = re.sub('\[.*?\]', '', text)             # Text in []
        text = re.sub('[Cc]horus', '', text)           # Chorus
        text = re.sub('x[0-9]+', '', text)             # Multiplikator (x5)
        text = re.sub('[0-9]+x', '', text)             # Multiplikator (5x)
        text = re.sub('[\"\.,!\?\-()\[\]]', ' ', text) # Sonderzeichen
        text = re.sub('\s+', ' ', text)                # Wiederholte Leerzeichen
        out.append(text)
    return out

### Noise Removal
Nicht relevante Informationen aus dem Text sollen entfernt werden. 

Sogenannter Noise kann mittels Abgleich von einer "stopwords"-Liste entfernt werden. In dieser Liste befinden sich Konjunktionen (Bindewörter), URLs oder Links, Hashtags und Verweise, Zeichensetzungen und industriespezifische Wörter. 

In [7]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

def removeCommonWords(items):
    out = []
    for text in items:
        newText = []
        words = text.split()
        for word in words:
            if word not in stoplist:
                newText.append(word)
        out.append(' '.join(newText))
    return out

### Lexicon Normalization

Wörter werden mittels Stemming und Lemmatization in ihre Grundform gesetzt, so werden die Texte untereinander vergleichbarer. 

>__Stemming:__ Worte in seine Grundform (Infinitiv) bringen, so können Texte besser miteinander verglichen werden. (Affected, Affection, Affecting -> Affect)

>__Lemmatization:__ Gruppieren von Wörter mit dem gleichen Ursprung (gone, going, went -> go)

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
wnlem = WordNetLemmatizer()
pstem = PorterStemmer()

def normalizeWords(items):
    out = []
    for text in items:
        newText = []
        words = text.split()
        for word in words:
            word = word.lower()
            word = wnlem.lemmatize(word, wordnet.VERB)
            word = pstem.stem(word)
            newText.append(word)
        out.append(' '.join(newText))
    return out

## Feature Generierung
Machine Learning Algorithmen können mit Text nichts anfangen, der Text muss zuerst in Features umgewandelt werden. Es gibt eine Vielzahl von Verfahren um dies zu tun. Als einfaches Beispiel könnte die Wortanzahl oder die mittlere Wortlänge als solch ein Feature verwendet werden. 

### Wortanzahl

In [9]:
df['text'].apply(lambda t: len(t.split())).head()

361    279
362    348
363    173
364    106
365    181
Name: text, dtype: int64

### Mittlere Wortlänge

In [10]:
df['text'].apply(
    lambda t: np.average([len(word) for word in t.split()])).head()

361    3.971326
362    3.706897
363    5.387283
364    5.254717
365    4.055249
Name: text, dtype: float64

### N-Grams

Es werden Wörter im Text gruppiert (bigrams, trigrams, etc.). Die Häufigkeit dieser Gruppen im Vergleich zu anderen Texten kann dann für die Auswertung verwendet werden. Die wichtigsten N Grams sind die Bigrams (N = 2).


In [11]:
def generateNGram(text, n):
    s = []
    words = text.split()
    for i in range(len(words)-n+1):
        s.append(words[i:i+n])
    return s

generateNGram('lorem ipsum dolor sit amet', 2) # Bigrams

[['lorem', 'ipsum'], ['ipsum', 'dolor'], ['dolor', 'sit'], ['sit', 'amet']]

### TF-IDF

Dieses Modell wandelt den Text in Vektoren um, die Länge der Vektoren bestimmt sich durch die Häufigkeit des Aufkommens von Wörtern. Die TF IDF ermittelt die relative Wichtigkeit von Wörtern in einem Text.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer 

print(TfidfVectorizer().fit_transform(
    ['this is sample document',
     'another random document',
     'third sample document text']))

  (0, 1)	0.34520501686496574
  (0, 4)	0.444514311537431
  (0, 2)	0.5844829010200651
  (0, 7)	0.5844829010200651
  (1, 3)	0.652490884512534
  (1, 0)	0.652490884512534
  (1, 1)	0.3853716274664007
  (2, 5)	0.5844829010200651
  (2, 6)	0.5844829010200651
  (2, 1)	0.34520501686496574
  (2, 4)	0.444514311537431


### Bag-of-Words

Für jeden Text werden alle vorkommenden Wörter notiert, diese Liste kann dan als Grundlage für den Vergleich mit anderen Texten verwendet werden. 

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(df[df['artist'] == 'Bob Dylan']['text'])
# Alle Wörter die vom Interpret "Bob Dylan" verwendet werden:
print(vectorizer.get_feature_names()) 

['08', '10', '100', '101', '102', '14', '15', '16', '17', '1792', '18', '19', '1910', '26', '27', '28', '29', '49', '51', '59', '61', '707', '99', 'abandon', 'abandoned', 'abe', 'abel', 'aberdeen', 'abiding', 'able', 'about', 'above', 'abraham', 'abuse', 'acapulco', 'accept', 'accordingly', 'accountant', 'accused', 'ace', 'achieve', 'achin', 'aching', 'acre', 'across', 'act', 'acting', 'acts', 'ad', 'addict', 'address', 'adjust', 'admit', 'adore', 'advance', 'advances', 'advertise', 'advertising', 'advice', 'aeroplane', 'affair', 'affairs', 'affection', 'affirmation', 'afford', 'afraid', 'after', 'aftermath', 'afternoon', 'again', 'against', 'age', 'agents', 'ago', 'agree', 'ah', 'ahead', 'ahold', 'aid', 'aime', 'aimin', 'ain', 'air', 'airport', 'aisles', 'alabama', 'aladdin', 'alive', 'all', 'alla', 'alley', 'alleys', 'alligator', 'allow', 'allowed', 'almighty', 'almost', 'alone', 'along', 'alphabet', 'already', 'alright', 'also', 'altars', 'alters', 'although', 'always', 'am', 'amazi

### Bag-of-N-grams
Hier gilt das gleiche Prinzip wie bei Bag of Words, mit dem Unterschied, dass hier vorkommende N-Grams verglichen werden.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

c_vec = CountVectorizer(ngram_range=(1, 3))
c_vec.fit(["an apple a day keeps the doctor away"]).vocabulary_

{'an': 0,
 'apple': 3,
 'day': 7,
 'keeps': 12,
 'the': 15,
 'doctor': 10,
 'away': 6,
 'an apple': 1,
 'apple day': 4,
 'day keeps': 8,
 'keeps the': 13,
 'the doctor': 16,
 'doctor away': 11,
 'an apple day': 2,
 'apple day keeps': 5,
 'day keeps the': 9,
 'keeps the doctor': 14,
 'the doctor away': 17}

## Pipeline
Die Pipeline besteht aus einer Liste (Name, Transformation), in welcher die einzelnen Einträge der Reihe nach abgearbeitet werden, in diesem Fall folgt zuerst das Preprozessing, dann die Feature-Generierung und zum Schluss die Klassifikation, welche die vorprozessierten Daten mittels den generierten Features abarbeitet. 

Die Klassifizierung besteht aus einem Lerner und einer Prediction. Dem Lerner werden 80 Prozent der Daten zur Verfügung gestellt. Dieser lernt dann mit den generierten Features und nutzt die Prediction mit den restlichen 20 Prozent, um eine Schätzung abzugeben.

In [15]:
from sklearn.base import TransformerMixin

class Preprocessor(TransformerMixin):
    def transform(self, X, **transform_params):
        X = clean(X)
        # Auskommentiert um die Berechnungen zu beschleunigen
        #X = removeCommonWords(X)
        #X = normalizeWords(X)
        return X

    def fit(self, X, y=None, **fit_params):
        return self

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

Pipeline([
    ('pre', Preprocessor()),

    # feature generation
    ('tfidf', TfidfVectorizer()),
    
    # classification
    ('knn', KNeighborsClassifier()),
]);

## Train-data, Target-data
Das Lernen und das Testen einer Funktion darf nicht mit den selben Daten geschehen, ein Modell, dass nur mit der Stichprobe getestet wird, kann nicht falsch liegen.

In [17]:
from sklearn.model_selection import train_test_split

X=df['text'].values
y=df['artist'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## No Free Lunch / Kreuzvalidierung

Mittels Kreuzvalidierung können die einzelnen Klassifikatoren und Feature Generatoren untereinander verglichen werden.

>Klassifikatoren:
- __SGDClassifier:__ (stochastic gradient descent) Nützlich bei hoher Anzahl Samples und Features
- __SVM:__ (support vector machines) Nützlich bei hoher Anzahl Samples und Features
- __DecisionTreeClassifier:__ Eine der ältestetn Techniken.
- __RandomForrestClassifier:__ Basiert auf einer Kombination von verschiedenen Entscheidungsbäumen.
- __MultinomialNB:__ Eignet sich für Textklassifikation.
- __KNeighborsClassifier:__ Untersucht k der nächsten Nachbarn.
- __LogisticRegression:__ Klassifikator auf Basis einer "S" geformten Kurve

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn import tree

featureGenerators = [
    [('tfidf', TfidfVectorizer())],
    [('bow', CountVectorizer(stop_words='english')),
     ('tfidf', TfidfTransformer())],
    [('bongrams', CountVectorizer(stop_words='english', ngram_range=(1, 3))),
     ('tfidf', TfidfTransformer())],
]

classifiers = [
    ('svc', svm.SVC(kernel='linear')),
    ('tree', tree.DecisionTreeClassifier()),
    ('forest', RandomForestClassifier(n_estimators=50)),
    ('mnb', MultinomialNB()),
    ('knn', KNeighborsClassifier()),
    ('sgd', SGDClassifier(tol=1e-3)),
    ('logreg', LogisticRegression(solver='lbfgs', multi_class='auto')),
]

pipelines = []
for fg in featureGenerators:
    for clas in classifiers:
        pipelines.append(Pipeline([
            ('preprocess', Preprocessor()),
            *fg,
            clas
        ]))

In [19]:
scores = pd.DataFrame(columns=['steps','accuracy_score'])

for p in pipelines:
    p.fit(X_train, y_train)
    scores.loc[len(scores)] = [
        [step[0] for step in p.steps],
        p.score(X_test, y_test)
    ]
    
scores.sort_values(by='accuracy_score', ascending=False, inplace=True)
print(scores.to_string(index=False))

                                 steps  accuracy_score
           [preprocess, tfidf, logreg]        0.308955
              [preprocess, tfidf, svc]        0.301493
              [preprocess, tfidf, sgd]        0.294030
      [preprocess, bow, tfidf, logreg]        0.273134
         [preprocess, bow, tfidf, svc]        0.268657
    [preprocess, bongrams, tfidf, sgd]        0.262687
    [preprocess, bongrams, tfidf, svc]        0.258209
         [preprocess, bow, tfidf, mnb]        0.256716
 [preprocess, bongrams, tfidf, logreg]        0.255224
         [preprocess, bow, tfidf, sgd]        0.250746
      [preprocess, bow, tfidf, forest]        0.241791
    [preprocess, bongrams, tfidf, mnb]        0.235821
              [preprocess, tfidf, mnb]        0.223881
 [preprocess, bongrams, tfidf, forest]        0.216418
           [preprocess, tfidf, forest]        0.198507
              [preprocess, tfidf, knn]        0.143284
        [preprocess, bow, tfidf, tree]        0.132836
          

## Entscheid

Die Kombination aus TF-IDF und der logistischen Regression erzielen unter den Testbedingungen (< 50 Klassen) stabil eine hohe Accuracy Score.

Die Parameter für den Klassifikator werden mit GridSearchCV getuned.

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

param_grid = {
    'logisticregression__C': [.1, 10, 100],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear']
}
logregModel = make_pipeline(
    Preprocessor(),
    TfidfVectorizer(),
    LogisticRegression(multi_class='auto', max_iter=200)
)
grid = GridSearchCV(logregModel, param_grid, cv=5, n_jobs=-1, return_train_score=True)

In [21]:
grid.fit(X, y);
grid.best_params_

/usr/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'logisticregression__C': 10, 'logisticregression__solver': 'newton-cg'}

In [22]:
model = make_pipeline(
    Preprocessor(),
    TfidfVectorizer(),
    LogisticRegression(multi_class='auto', C=10, solver='newton-cg', max_iter=200)
)

## Validierung

In [23]:
from sklearn.metrics import classification_report

model.fit(X_train, y_train)

prediction = model.predict(X_test)
print(classification_report(y_test, prediction))

                   precision    recall  f1-score   support

          Alabama       0.23      0.29      0.26        31
          America       0.23      0.28      0.25        29
        Bob Dylan       0.24      0.30      0.27        33
       Chaka Khan       0.29      0.24      0.26        38
             Cher       0.23      0.21      0.22        34
    Cliff Richard       0.05      0.06      0.06        33
      Dean Martin       0.39      0.33      0.36        46
     Donna Summer       0.29      0.25      0.27        40
    George Strait       0.29      0.29      0.29        34
 Gordon Lightfoot       0.34      0.36      0.35        42
Hank Williams Jr.       0.50      0.38      0.43        45
     Indigo Girls       0.44      0.42      0.43        40
      Johnny Cash       0.29      0.25      0.27        40
             Kiss       0.39      0.45      0.42        40
     Loretta Lynn       0.24      0.22      0.23        41
         Nazareth       0.29      0.43      0.34       

In [24]:
# Neil Young (Text aus neuem Album, das nicht im Datensatz vorhanden ist.)
text = ["""
Up in the rainbow teepee sky No one's looking down on you or I That's just a mirror in your eye
Ain't taken my last hit yet I know that things are different now (I see the same old signs, but something new is growing)
Don't think I'll cash it in yet Don't think I'll put down my last bet (I'm gonna keep my hand in, because something new is growing)
Think I'll hit the Peace Trail Take a trip back home to my old town 'Cause everyone back there says Something new is growing
Up in the rainbow teepee sky No one's looking down on you or I It's just a mirror in our eye
If I believe in someone I have to believe in myself (I have to take good care when something new is growing)
The world is full of changes Sometimes all these changes make me sad (I have to plant them seeds, till something new is growing)
I think I'll hit the Peace Trail I know that treasure takes its time (I have to take good care when something new is growing)
I think I'll hit the Peace Trail I think I like my chances now (I have to take good care when something new is growing)
I think I'll hit the Peace Trail I think I'll hit the Peace Trail now Because something new is growing 
"""]
model.predict(text)

array(['Neil Young'], dtype=object)